In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import math
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import v2
from torch.utils.data import random_split
from tqdm.auto import tqdm

class CNNModel(nn.Module):
    def outsize(self, input_size, kernel_size, padding, stride):
        return math.floor(input_size + 2 * padding - (kernel_size -1) -1) / stride + 1
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.2)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(3136, 64)
        self.linear2 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = F.relu(self.linear1(x))
        output = self.linear2(x)

        return output

def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(train_loader, leave=False, disable=False):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        preds = model(inputs)
        loss = criterion(preds, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        max_scores, predicted = torch.max(preds, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        

    train_loss = running_loss / total
    train_acc = correct / total
    return train_loss, train_acc


def evaluate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, leave=False, disable=False):
            images, labels = images.to(device), labels.to(device)
            preds = model(images)
            loss = criterion(preds, labels)

            running_loss += loss.item() * images.size(0)
            max_scores, predicted = torch.max(preds, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    val_loss = running_loss / total
    val_acc = correct / total
    return val_loss, val_acc



train_tensor = torchvision.datasets.FashionMNIST(root='./fashion_mnist', train=True, download=True,
                                            transform=v2.ToTensor())
test_tensor = torchvision.datasets.FashionMNIST(root='./fashion_mnist', train=False, download=True,
                                           transform=v2.ToTensor())

train_imgs = torch.stack([img for img, _ in train_tensor], dim=0)
#train_mean = train_imgs.mean(dim=(0, 2, 3)).tolist() # RGB
#train_std = train_imgs.std(dim=(0, 2, 3)).tolist() # RGB
train_mean = [train_imgs.mean().item()]
train_std = [train_imgs.std().item()]

transform_train = v2.Compose(
    [
        v2.ToImage(),
        v2.RandomHorizontalFlip(),
        v2.RandomResizedCrop(size=28),
        v2.RandomRotation(degrees=10),
        v2.ToDtype(dtype=torch.float32, scale=True),
        v2.Normalize(mean=train_mean, std=train_std),
    ]
)

transform_test = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(dtype=torch.float32, scale=True),
        v2.Normalize(mean=train_mean, std=train_std)
    ]
)
 
def data_load():
    train_data = datasets.FashionMNIST(root='./fashion_mnist', train=True, download=True, transform=transform_train)
    test_data = datasets.FashionMNIST(root='./fashion_mnist', train=False, download=True, transform=transform_test)
    train_data = torch.utils.data.Subset(train_data, range(100)) # 테스트 100개

    print('학습데이터셋 크기 ', len(train_data))
    print('테스트데이터셋 크기 ', len(test_data))

    img, label = train_data[0]
    print(img.shape)  # (채널, 높이, 너비)
    
    train_dataset = train_data
    #val_dataset, test_dataset = random_split(test_data, [5000, 5000])
    val_dataset, test_dataset = random_split(test_data, [50, len(test_data)-50]) # 테스트 50개

    train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    return train_dataloader, val_dataloader, test_dataloader

train_dataloader, val_dataloader, test_dataloader = data_load()
print('학습데이터셋 크기 ', len(train_dataloader))
print('테스트데이터셋 크기 ', len(test_dataloader))

model = CNNModel()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

print('Training start!')

epochs = 5
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_dataloader, loss_fn, optimizer, device)
    val_loss, val_acc = evaluate(model, val_dataloader, loss_fn, device)
    print(f'epoch {epoch+1}/{epochs}, val loss: {val_loss:.4f}, val acc: {val_acc:.4f}')
    
print('Training finished!')

학습데이터셋 크기  100
테스트데이터셋 크기  10000
torch.Size([1, 28, 28])
학습데이터셋 크기  1
테스트데이터셋 크기  311
Training start!


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

epoch 1/5, val loss: 2.2712, val acc: 0.2000


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

epoch 4/5, val loss: 2.2312, val acc: 0.1400


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

epoch 5/5, val loss: 2.1191, val acc: 0.1800
Training finished!
